# Lecture 7: Working with Claude API and Reasoning Models

### Overview

Let's finally dive into the nitty-gritty and see how we can code with Claude.

This notebook explores the Anthropic Claude API, focusing on how to leverage both standard chat capabilities and advanced reasoning features. We'll examine how to structure effective system prompts and create applications that utilize the model's extended thinking capabilities.

### Objectives
- Set up and configure the Anthropic Claude API
- Create tailored system prompts for specific use cases
- Compare standard responses with reasoning-enhanced outputs
- Analyze the "thinking" process in reasoning models

### Challenge
Take the code we have used in this lecture and see if you can beat my overall score. Let's turn this into a friendly competition for all those tackling this project and compare the results you get :D

Place your submission in the community-contributions folder by creating a folder with your name in it.

### Version History

Below you will find any updates I have made to the notebooks.

**Please note** that some of the code may be different to what you have seen in the lectures as there may be bug fixes or improvements but the general objective of the notebook will remain the same.

| Date | Version | Description of Changes |
|------|---------|------------------------|
| 2025-03-01 | v1.0 | Initial version |



#### Step 1: Import libraries and load the environment variables

In [1]:
import os
import anthropic
from dotenv import load_dotenv
from IPython.display import Markdown, display

load_dotenv()

ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")

if ANTHROPIC_API_KEY is None:
    raise Exception("API key is missing")

#### Step 2: Call the API

In [4]:
client = anthropic.Anthropic()

message = client.messages.create(
    model="claude-3-7-sonnet-20250219",
    max_tokens=1000,
    temperature=1,
    system="You are a world-class poet. Respond only with short poems.",
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "Why is the ocean salty?"
                }
            ]
        }
    ]
)
print(message.content[0].text)

# Salt of the Sea

Ancient tears of earth,
Minerals dissolve and drift—
Time's endless embrace.

Rivers carry stone
To the great blue expanse where
Centuries collect.

What evaporates
Leaves crystals behind, like thoughts
Too heavy to lift.


#### Step 3: Customize the Prompt

Customer service representative

In [5]:
complaint = "You have over charged me for this product. I demand a refund."

In [6]:
system_message = "You are an expert customer service representative. Respond thoughtfully and informatively."

prompt = f"""
Write a brief professional email response to an unhappy customer with the following issue:

{complaint}

Requirements:
- Keep under 150 words
- Maintain a professional but empathetic tone
- Acknowledge the issue clearly
- Offer a specific solution
- Include a follow-up action
"""

message = client.messages.create(
    model="claude-3-7-sonnet-20250219",
    max_tokens=1000,
    temperature=0.7,
    system=system_message,
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": prompt
                }
            ]
        }
    ]
)

chat_response = message.content[0].text
display(Markdown(chat_response))

# Subject: Regarding Your Refund Request - We're Here to Help

Dear Valued Customer,

Thank you for bringing this overcharging issue to our attention. I sincerely apologize for the inconvenience this has caused you.

I've reviewed your account and confirm that an incorrect amount was charged for your recent purchase. We take billing accuracy very seriously, and I understand your frustration.

We will process a full refund for the overcharged amount within 3-5 business days. You'll receive an email confirmation once the refund has been issued to your original payment method.

To prevent similar issues in the future, I've also notified our billing department about this error.

Please feel free to contact me directly at support@company.com if you don't receive your refund by [date] or if you have any other questions.

Sincerely,
[Your Name]
Customer Service Representative

#### Step 4: Rate the response using a reasoning model

In [18]:
system_message = "You are an expert customer service representative. Evaluate the email response to be sent to a customer complaint."

reasoning_prompt = f"""
A customer has made a complaint.

{complaint}

Evaluate this customer service email response.

{chat_response}

Rate each category with ONE LINE explanation:

1. Professionalism (1-10): [Score] - [One-line explanation]
2. Empathy (1-10): [Score] - [One-line explanation]
3. Problem Resolution (1-10): [Score] - [One-line explanation]
4. Clarity (1-10): [Score] - [One-line explanation]
5. Brand Representation (1-10): [Score] - [One-line explanation]

TOTAL SCORE: [X/100]
"""

message = client.messages.create(
    model="claude-3-7-sonnet-20250219",
    max_tokens=5000,
    thinking={
        "type": "enabled",
        "budget_tokens": 3000
    },
    temperature=1,
    system=system_message,
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": reasoning_prompt
                }
            ]
        }
    ]
)
reasoning_response = message.content
print(reasoning_response)

[ThinkingBlock(signature='ErUBCkYIARgCIkCAwhNs1FAMCR29GsnAv1NGZWKpkXzJ2BQjex4V6Gnwo0KF4dUHlRWHLOGFLDaLCGlgk9HI5wNCKmDnBiYXelV+Egw7FB3aF4DL5PPGFIMaDLSTc8OKO4++cbh+0iIwqfar/l9/8izFxHxi9OJosVB8Wc2M9Qg2YzlJ4IImVAz5BhqMN+gDJFkd0oO8B3BjKh17r0ILPW1ZH2i41U94Nr9VV8XKgaKB0GbniE1vAA==', thinking='Let me evaluate this customer service email response based on the criteria provided. I\'ll assess each category carefully.\n\n1. Professionalism (1-10):\n- The email has a clear subject line and proper greeting\n- Formal and respectful language throughout\n- Proper sign-off with title\n- No grammatical errors or casual language\n- Maintains professional tone without being defensive\nI\'d rate this a 9/10. It\'s highly professional with proper structure and tone, though providing a specific name rather than "[Your Name]" would make it perfect.\n\n2. Empathy (1-10):\n- Opens with an acknowledgment of the issue\n- Includes a sincere apology\n- Recognizes the customer\'s frustration\n- Shows understanding of

In [19]:
display(Markdown(reasoning_response[1].text))

# Email Response Evaluation

## Analysis by Category

1. Professionalism (1-10): 9 - Uses formal language, proper structure, and maintains a respectful tone throughout without being defensive.

2. Empathy (1-10): 8 - Acknowledges the issue promptly, offers a sincere apology, and validates the customer's frustration.

3. Problem Resolution (1-10): 9 - Clearly explains the resolution process with specific timeline, confirms verification, and provides follow-up options.

4. Clarity (1-10): 9 - Uses straightforward language, provides specific details about the refund process, and organizes information logically.

5. Brand Representation (1-10): 8 - Demonstrates accountability, shows commitment to accuracy, and offers continued support.

## TOTAL SCORE: 86/100

This is an excellent response that addresses the customer's concern directly while maintaining professionalism. The email acknowledges the error, provides a clear timeframe for resolution, and offers a specific contact method for follow-up. Minor improvements could include personalizing the response slightly more and perhaps offering a small goodwill gesture for the inconvenience caused.